In [1]:
import laspy
import numpy as np
from scipy.spatial import KDTree
from user_inputs import laz_file_path, clip_folder_path
from user_inputs import search_radius, nn_threshold, bin_count_nn
from helper_functions import generate_bins, fraction_of_trues
import time
from tqdm import tqdm

In [2]:
laz_file_name = laz_file_path.split("\\")[-1].split(".")
clip_laz_file_name = laz_file_name[0] + "_clipped_8th_part." + laz_file_name[1]
clip_laz_path = clip_folder_path + "\\" + clip_laz_file_name
clip_laz_path

'D:\\Geomatics\\thesis\\RefineNet\\data\\AHN\\clipped\\C_37EN2_clipped_8th_part.LAZ'

In [3]:
laz = laspy.read(clip_laz_path)

In [4]:
# number of points
print(f"number of points: {laz.header.point_count}")

number of points: 18742173


In [5]:
# printing out all the available dimensions in clipped laz file
for dimension in laz.point_format.dimensions:
    print(dimension.name)
print("\nunique classifications in clipped region: ", np.unique(laz.classification))

X
Y
Z
intensity
return_number
number_of_returns
scan_direction_flag
edge_of_flight_line
classification
synthetic
key_point
withheld
scan_angle_rank
user_data
point_source_id
gps_time
Amplitude
Reflectance
Deviation

unique classifications in clipped region:  [ 1  2  6  9 26]


**Neighbourhood search steps**
- making a kd tree (first load the points)
- finding the nearest points within 0.5 m radius
    - we can first build np array (or pandas dataframe) with all the required points, classification (labels), rgb(?), intensities
    - then use np.where to find out the distances and indexes of the neighbour points
    - filter only those within 0.5m and look for their classifications
    - give confidence scores to points
    - add extra dimension to laz file

In [6]:
# loading the xyz values
xyz = laz.xyz.copy()
print("xyz shape: ", xyz.shape)
# build a kd tree
tree = KDTree(xyz)

xyz shape:  (18742173, 3)


In [8]:
# loading the classifications of points
labels = laz.classification
print("labels shape: ", labels.shape)

labels shape:  (18742173,)


### **Binning the data to take off load from kd tree**

- number of bins is a hyper-parameter, can be modified in `user_inputs.py`

In [9]:
# binning for neighbourhood search, nobebook crashing when given all the points
num_points = laz.header.point_count
nn_bins = generate_bins(number=num_points, bin_count=bin_count_nn)
print("num_points: ", num_points)
print("number of bins: ", bin_count_nn)
print(nn_bins)

num_points:  18742173
number of bins:  40
[[0, 468554], [468554, 937109], [937109, 1405663], [1405663, 1874217], [1874217, 2342772], [2342772, 2811326], [2811326, 3279880], [3279880, 3748435], [3748435, 4216989], [4216989, 4685543], [4685543, 5154098], [5154098, 5622652], [5622652, 6091206], [6091206, 6559761], [6559761, 7028315], [7028315, 7496869], [7496869, 7965424], [7965424, 8433978], [8433978, 8902532], [8902532, 9371086], [9371086, 9839641], [9839641, 10308195], [10308195, 10776749], [10776749, 11245304], [11245304, 11713858], [11713858, 12182412], [12182412, 12650967], [12650967, 13119521], [13119521, 13588075], [13588075, 14056630], [14056630, 14525184], [14525184, 14993738], [14993738, 15462293], [15462293, 15930847], [15930847, 16399401], [16399401, 16867956], [16867956, 17336510], [17336510, 17805064], [17805064, 18273619], [18273619, 18742173]]


#### *Looping throung all the bins sequentially and calculate confidences*  
- **Get all the neighbouring points (including itself) for a given point**
- seperate the current point from the neighbours
- get classification of the current point
- get classification of the neighbour points
- **Calculate confidences**
    - **count the percentage of neighbour points are of same classification**
    - **points with neighbours < threshold => we would be giving `0` confidence.. should be altered later**

**Confidence = (label_match_count) / (neighbour_count)**
1. get match count
2. get neighbour count
3. calculate confidence

**After**
1. get threshold mask
2. less than 5 threshold number of neighbours, give confidence of `0`

In [10]:
confidence_scores = []
count = 0
point_count = 0
for bin in tqdm(nn_bins, desc="Processing bins"):

    # if count != 15:
    #     count += 1
    #     continue
    # else:
    #     count += 1

    # the query points numbers and their labels    
    start, end = bin[0], bin[1]
    pts = np.arange(start, end, step=1)
    pts_vector = pts.reshape((-1, 1))
    pts_labels = np.array(labels[pts])
    pts_labels = pts_labels.reshape((-1, 1))  # reshaping it to a column vector -> easy to make comparison with neighbours

    # neighbours information of query points
    nn = tree.query_ball_point(xyz[start:end, :], r=0.5, p=2, workers=-1, return_sorted=True)

    #   # should vectorize calculations so find the query point with max number of elements
    max_length = max(len(lst) for lst in nn)
    nn_np_padded = np.array([lst + [np.nan] * (max_length - len(lst)) for lst in nn], dtype=object)  # self points should be removed
    nn_np_padded_float = nn_np_padded.astype(float)  # self points should be removed

    print("pts_vector[250]: ", pts_vector[250])
    print("nn_np_padded[250]: ", nn_np_padded[250])

    # remove the search point itself from the neighbours by creating a mask
    self_mask = (pts_vector == nn_np_padded_float)
    nn_np_padded[self_mask] = np.nan
    nn_np_padded_float[self_mask] = np.nan

    print("nn_np_padded[250]: ", nn_np_padded[250])

    # flattening the neighbours array 
    # so that it is easy to pass to labels and get the classificaitons
    nn_np_padded_flat = nn_np_padded.reshape((1, -1))
    nn_np_padded_float_flat = nn_np_padded_float.reshape((1, -1))
    # nanMask ## True when the element is nan
    nn_np_padded_float_flat_nanMask = np.isnan(nn_np_padded_float_flat)
    nn_np_padded_Mask = ~np.isnan(nn_np_padded_float)

    # replacing all the nan's with -1 index as a placeholder, later we can use mask again to filter them out 
    nn_np_padded_flat[nn_np_padded_float_flat_nanMask] = -1  # replacing all the nan's with -1, now we can use this variable to get neighbours classifications
    nn_labels_flat_pHold = labels[list(nn_np_padded_flat[0])]
    nn_labels_flat_pHold = np.array(nn_labels_flat_pHold)
    # nn_labels_flat = nn_labels_flat_pHold *  ~nn_np_padded_float_flat_nanMask
    nn_labels_flat = nn_labels_flat_pHold.copy()
    nn_labels_flat_float = nn_labels_flat.astype(float)
    nn_labels_flat_float[nn_np_padded_float_flat_nanMask[0]] = np.nan

    # 2d array => no more flattened
    nn_labels_float = nn_labels_flat_float.reshape((nn_np_padded.shape[0], -1))


    # 1. compare pts_labels with nn_labels
    match_pts_nn_bool = (pts_labels == nn_labels_float)
    match_pts_nn_num = np.sum(match_pts_nn_bool, axis=1, keepdims=True) # number of points that match

    # 2. nn_count, create nn mask to filter out the points which are not surrounded by many points
    nn_count = np.sum(nn_np_padded_Mask, axis=1, keepdims=True)
    nn_threshold_mask = (nn_count < nn_threshold)

    # 3. calculating the confidences
    confidences = match_pts_nn_num / nn_count 
    confidences[nn_threshold_mask] = 0
    
    # 4. combining all the bins together
    confidence_scores.append(confidences)

    print("start, end: ", start, end)
    print("pts_vector.shape: ", pts_vector.shape)
    print("pts_vector: ", pts_vector)
    print("nn.shape: ", nn.shape)
    # print()


confidence_scores = np.vstack(confidence_scores)
print(confidence_scores.shape)    

Processing bins:   0%|          | 0/40 [00:00<?, ?it/s]

pts_vector[250]:  [250]
nn_np_padded[250]:  [250 63248 63978 63979 63980 2629218 2629219 2629220 2629907 2629908
 2629909 2630609 2630610 nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
nn_np_padded[250]:  [nan 63248 63978 63979 63980 2629218 2629219 2629220 2629907 2629908
 2629909 2630609 2630610 nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]


C:\conda_tmp\ipykernel_25584\241511297.py:67: RuntimeWarning: invalid value encountered in divide
  confidences = match_pts_nn_num / nn_count
Processing bins:   2%|▎         | 1/40 [00:03<02:31,  3.88s/it]

start, end:  0 468554
pts_vector.shape:  (468554, 1)
pts_vector:  [[     0]
 [     1]
 [     2]
 ...
 [468551]
 [468552]
 [468553]]
nn.shape:  (468554,)
pts_vector[250]:  [468804]
nn_np_padded[250]:  [466028 466029 466030 467420 467421 467422 468803 468804 468805 470197
 470198 470199 3061974 3061975 3061976 3063020 3063021 3063022 3063023
 3064066 3064067 3064068 nan nan nan nan nan nan]
nn_np_padded[250]:  [466028 466029 466030 467420 467421 467422 468803 nan 468805 470197 470198
 470199 3061974 3061975 3061976 3063020 3063021 3063022 3063023 3064066
 3064067 3064068 nan nan nan nan nan nan]


Processing bins:   5%|▌         | 2/40 [00:08<02:34,  4.06s/it]

...

start, end:  17336510 17805064
pts_vector.shape:  (468554, 1)
pts_vector:  [[17336510]
 [17336511]
 [17336512]
 ...
 [17805061]
 [17805062]
 [17805063]]
nn.shape:  (468554,)
pts_vector[250]:  [17805314]
nn_np_padded[250]:  [16931542 16931543 16931544 16932474 16932475 16932476 16933396 16933397
 16933398 17802575 17802576 17803936 17803937 17803938 17805313 17805314
 17805315 17806683 17806684 17806685 nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan]
nn_np_padded[250]:  [16931542 16931543 16931544 16932474 16932475 16932476 16933396 16933397
 16933398 17802575 17802576 17803936 17803937 17803938 17805313 nan
 17805315 17806683 17806684 17806685 nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan]


Processing bins:  98%|█████████▊| 39/40 [03:14<00:04,  4.98s/it]

start, end:  17805064 18273619
pts_vector.shape:  (468555, 1)
pts_vector:  [[17805064]
 [17805065]
 [17805066]
 ...
 [18273616]
 [18273617]
 [18273618]]
nn.shape:  (468555,)
pts_vector[250]:  [18273869]
nn_np_padded[250]:  [17190927 17190928 17191816 17191817 17191818 17192731 17192732 17192733
 17193637 17193638 17193639 18273272 18273578 18273579 18273580 18273868
 18273869 18273870 18274156 18274157 18274158 nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
nn_np_padded[250]:  [17190927 17190928 17191816 17191817 17191818 17192731 17192732 17192733
 17193637 17193638 17193639 18273272 18273578 18273579 18273580 18273868
 nan 18273870 18274156 18274157 18274158 nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]


Processing bins: 100%|██████████| 40/40 [03:20<00:00,  5.01s/it]

start, end:  18273619 18742173
pts_vector.shape:  (468554, 1)
pts_vector:  [[18273619]
 [18273620]
 [18273621]
 ...
 [18742170]
 [18742171]
 [18742172]]
nn.shape:  (468554,)
(18742173, 1)


In [ ]:
confidence_scores

## Useless

In [ ]:
confidence_scores = []
count = 0
point_count = 0

for bin in tqdm(nn_bins, desc="Processing bins"):

    # the query points numbers and their labels 
    start, end = bin[0], bin[1]
    pts = np.arange(start, end, step=1)
    pts_vector = pts.reshape((-1, 1))
    pts_labels = np.array(labels[pts])
    pts_labels = pts_labels.reshape((-1, 1))  # reshaping it to a column vector -> easy to make comparison with neighbours

    point_count += pts_vector.shape[0]


    # # neighbours information of query points
    # nn = tree.query_ball_point(xyz[start:end, :], r=0.5, p=2, workers=-1, return_sorted=True)

    # #   # should vectorize calculations so find the query point with max number of elements
    # max_length = max(len(lst) for lst in nn)
    # nn_np_padded = np.array([lst + [np.nan] * (max_length - len(lst)) for lst in nn], dtype=object)  # self points should be removed
    # nn_np_padded_float = nn_np_padded.astype(float)  # self points should be removed

    # print("pts_vector[250]: ", pts_vector[250])
    # print("nn_np_padded[250]: ", nn_np_padded[250])

    # # remove the search point itself from the neighbours by creating a mask
    # self_mask = (pts_vector == nn_np_padded_float)
    # nn_np_padded[self_mask] = np.nan
    # nn_np_padded_float[self_mask] = np.nan

    # print("nn_np_padded[250]: ", nn_np_padded[250])

    # # flattening the neighbours array 
    # # so that it is easy to pass to labels and get the classificaitons
    # nn_np_padded_flat = nn_np_padded.reshape((1, -1))
    # nn_np_padded_float_flat = nn_np_padded_float.reshape((1, -1))
    # # nanMask ## True when the element is nan
    # nn_np_padded_float_flat_nanMask = np.isnan(nn_np_padded_float_flat)
    # nn_np_padded_Mask = ~np.isnan(nn_np_padded_float)

    # # replacing all the nan's with -1 index as a placeholder, later we can use mask again to filter them out 
    # nn_np_padded_flat[nn_np_padded_float_flat_nanMask] = -1  # replacing all the nan's with -1, now we can use this variable to get neighbours classifications
    # nn_labels_flat_pHold = labels[list(nn_np_padded_flat[0])]
    # nn_labels_flat_pHold = np.array(nn_labels_flat_pHold)
    # # nn_labels_flat = nn_labels_flat_pHold *  ~nn_np_padded_float_flat_nanMask
    # nn_labels_flat = nn_labels_flat_pHold.copy()
    # nn_labels_flat_float = nn_labels_flat.astype(float)
    # nn_labels_flat_float[nn_np_padded_float_flat_nanMask[0]] = np.nan

    # # 2d array => no more flattened
    # nn_labels_float = nn_labels_flat_float.reshape((nn_np_padded.shape[0], -1))


    # # 1. compare pts_labels with nn_labels
    # match_pts_nn_bool = (pts_labels == nn_labels_float)
    # match_pts_nn_num = np.sum(match_pts_nn_bool, axis=1, keepdims=True) # number of points that match

    # # 2. nn_count, create nn mask to filter out the points which are not surrounded by many points
    # nn_count = np.sum(nn_np_padded_Mask, axis=1, keepdims=True)
    # nn_threshold_mask = (nn_count < nn_threshold)

    # # 3. calculating the confidences
    # confidences = match_pts_nn_num / nn_count 
    # confidences[nn_threshold_mask] = 0
    


    # print("start, end: ", start, end)
    # print("pts_vector.shape: ", pts_vector.shape)
    # print("pts_vector: ", pts_vector)
    # print("nn.shape: ", nn.shape)
    # # print()


    # 
print(f"point_count: {point_count}")
    

In [ ]:
laz.add_extra_dim(laspy.ExtraBytesParams(
    name="confidence",
    type=np.float64,
    description="Confidence of points"
))